# Visualize GEDI L2B and ICESat-2 ATL08 data with `lonboard`

## Import packages

In [1]:
#!pip install -U h5coro
!pip show h5coro # should be 0.0.7

Name: h5coro
Version: 0.0.7
Summary: Python package for reading HDF5 data from S3
Home-page: https://github.com/SlideRuleEarth/h5coro/
Author: SlideRule Developers
Author-email: 
License: BSD 3-Clause
Location: /srv/conda/envs/notebook/lib/python3.12/site-packages
Requires: boto3, earthaccess, numpy, requests
Required-by: 


In [2]:
pip install -e ../..

Obtaining file:///home/jovyan/hdf5_to_arrow
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for hdf5_to_arrow (pyproject.toml) ... done
  Created wheel for hdf5_to_arrow: filename=hdf5_to_arrow-0.0.0-0.editable-py3-none-any.whl size=2353 sha256=0bb2dfe91f0c0ca28d4387c63023e8ec2aa43f867b7dfedf32df34298af65804
  Stored in directory: /tmp/pip-ephem-wheel-cache-xmm246pb/wheels/ae/ce/01/c1fb4e4c6de38effbec8721d370319faad116328643507e268
Successfully built hdf5_to_arrow
  Attempting uninstall: hdf5_to_arrow
    Found existing installation: hdf5_to_arrow 0.0.0
    Uninstalling hdf5_to_arrow-0.0.0:
      Successfully uninstalled hdf5_to_arrow-0.0.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datetime import datetime, timezone, timedelta
import earthaccess

In [4]:
import sys
import os
from hdf5_to_arrow import hdf5_to_arrow

# Generate credentials

In [5]:
earthaccess.login()

Enter your Earthdata Login username:  aimeeb
Enter your Earthdata password:  ········


In [6]:
nsidc_aws_creds = earthaccess.get_s3_credentials(daac='NSIDC')
nsidc_credentials = dict(
    aws_access_key_id=nsidc_aws_creds['accessKeyId'],
    aws_secret_access_key=nsidc_aws_creds['secretAccessKey'],
    aws_session_token=nsidc_aws_creds['sessionToken']
)
lpdaac_aws_creds = earthaccess.get_s3_credentials(daac='LPDAAC')
lpdaac_credentials = dict(
    aws_access_key_id=lpdaac_aws_creds['accessKeyId'],
    aws_secret_access_key=lpdaac_aws_creds['secretAccessKey'],
    aws_session_token=lpdaac_aws_creds['sessionToken']
)

# Query for data uris (S3 URLs)

In [7]:
start = datetime(2021, 11, 1, tzinfo=timezone.utc)
end = start + timedelta(days=7)

atl08_results = earthaccess.search_data(
    short_name="ATL08",
    cloud_hosted=True,
    temporal=(start, end),
    bounding_box=(-90,-56,-32,14),
    count=-1
)
atl08_links = [result.data_links(access="direct")[0] for result in atl08_results]

In [8]:
gedil2b_results = earthaccess.search_data(
    short_name="GEDI02_B",
    cloud_hosted=True,
    temporal=(start, end),
    bounding_box=(-90,-56,-32,14),
    count=-1
)
gedil2b_links = [result.data_links(access="direct")[0] for result in gedil2b_results]

# Construct dataset arguments

In [9]:
atl08_variables = {
    "variables": {
        "h_canopy": {"dataset": "/gt1l/land_segments/canopy/h_canopy" },
        "dem_h": {"dataset": "/gt1l/land_segments/dem_h" }
    },
    "latitude": {"dataset": "/gt1l/land_segments/latitude" },
    "longitude": {"dataset": "/gt1l/land_segments/longitude" }
}

gedil2b_variables = {
    "variables": {
        "cover": {"dataset": "BEAM0000/cover" }
    },
    "latitude": {"dataset": "/BEAM0000/geolocation/lat_highestreturn" },
    "longitude": {"dataset": "/BEAM0000/geolocation/lon_highestreturn" }
}

# Create table for 1 file

Just to make sure it works.

In [10]:
creator = hdf5_to_arrow.HDF5ArrowTableCreator(
    uri=atl08_links[0],
    datasets=atl08_variables,
    credentials=nsidc_credentials,
    mask_using='h_canopy'
)
table = creator.create_table()

1727237160.924091 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values


# Create table for all files

In [11]:
%%time
gedil2b_table = hdf5_to_arrow.concat_tables(
    uris=gedil2b_links,
    datasets=gedil2b_variables,
    credentials=lpdaac_credentials,
    mask_using='cover'
)

only handle 1-dimensional arrays
only handle 1-dimensional arrays
CPU times: user 87.7 ms, sys: 95.4 ms, total: 183 ms
Wall time: 46.7 s


In [12]:
%%time
atl08_table = hdf5_to_arrow.concat_tables(
    uris=atl08_links,
    datasets=atl08_variables,
    credentials=nsidc_credentials,
    mask_using='h_canopy'
)

1727237211.045595 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1727237211.221663 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1727237211.410656 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1727237211.465480 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1727237212.241511 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1727237212.500017 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/DIMENSION_LIST is an unsupported datatype 7: unable to populate values
1727237212.745087 WARNING [h5dataset.py:  277] gt1l/land_segments/canopy/h_canopy/

only handle 1-dimensional arrays
CPU times: user 96.4 ms, sys: 43.8 ms, total: 140 ms
Wall time: 27 s


# Visualize with lonboard

In [13]:
# how many points?
gedil2b_table['cover'].length() + atl08_table['h_canopy'].length()

3029759

In [14]:
import lonboard

atl08_layer = lonboard.ScatterplotLayer(table=atl08_table)
gedil2b_layer = lonboard.ScatterplotLayer(table=gedil2b_table)

/srv/conda/envs/notebook/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [15]:
m = lonboard.Map([atl08_layer, gedil2b_layer])
m

Map(layers=[ScatterplotLayer(table=pyarrow.Table
dem_h: float
h_canopy: float
geometry: fixed_size_list<item: …